In [322]:
# Install the required packages
!pip install scikit-learn matplotlib pandas openpyxl

# Set labels for each document

In [323]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [324]:
def get_similarity(query: str, document: list[str]) -> float:
    document_tf_idf = TfidfVectorizer().fit_transform(document)
    query_tf_idf = TfidfVectorizer().fit(document)
    query_tf_idf = query_tf_idf.transform([query])
    
    cosine_similarities = cosine_similarity(query_tf_idf, document_tf_idf).flatten()
    return max(cosine_similarities)

In [325]:
# Load the datasets and use the keys to store the query
sheets = {}
queries = []
with pd.ExcelFile('data/loinc_dataset-v2.xlsx') as xls:
    queries.extend(xls.sheet_names)
    for query in queries:
        sheets[query] = pd.read_excel(xls, query, skiprows=2, header=0, index_col=0)
        
sheets[queries[0]].head(10)

,long_common_name,component,system,property
loinc_num,,,,
1988-5,C reactive protein [Mass/volume] in Serum or P...,C reactive protein,Ser/Plas,MCnc
1959-6,Bicarbonate [Moles/volume] in Blood,Bicarbonate,Bld,SCnc
10331-7,Rh [Type] in Blood,Rh,Bld,Type
18998-5,Trimethoprim+Sulfamethoxazole [Susceptibility],Trimethoprim+Sulfamethoxazole,Isolate,Susc
1975-2,Bilirubin.total [Mass/volume] in Serum or Plasma,Bilirubin,Ser/Plas,MCnc
890-4,Blood group antibody screen [Presence] in Seru...,Blood group antibody screen,Ser/Plas,ACnc
20565-8,"Carbon dioxide, total [Moles/volume] in Blood",Carbon dioxide,Bld,SCnc
18906-8,Ciprofloxacin [Susceptibility],Ciprofloxacin,Isolate,Susc
2143-6,Cortisol [Mass/volume] in Serum or Plasma,Cortisol,Ser/Plas,MCnc


In [326]:
# Compute the similarity for each query
for query in queries:
    similarities = pd.Series([], dtype=float)
    for index in sheets[query].index:
        value = sheets[query].loc[index].values.flatten()
        similarities[index] = get_similarity(query, value)
    sheets[query]['similarity'] = similarities

In [327]:
# Save the results to a new file
with pd.ExcelWriter('data/loinc_dataset-v2_similarity.xlsx') as writer:
    for query in queries:
        sheets[query].to_excel(writer, sheet_name=query, startrow=2, startcol=0, index=True)

# Train a model to predict the similarity

In [328]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error

In [329]:
seed = 42

In [330]:
models = {}
for query in queries:
    X = sheets[query].drop(columns=['similarity'])
    y = sheets[query]['similarity']
    
    # For each row, encode the value of each column
    X_encoded = pd.DataFrame()
    for column in X.columns:
        encoder = LabelEncoder()
        X_encoded[column] = encoder.fit_transform(X[column])
        
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=seed)
    
    # Train the model
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    # Predict the similarity
    y_pred = model.predict(X_test)
    
    # Evaluate the model
    error = root_mean_squared_error(y_test, y_pred)
    
    print(pd.DataFrame({'y_test': y_test, 'y_pred': y_pred}).head(10))
    
    models[query] = model
    print(f'Model for query = \"{query}\" obtains an error: {error:.2f}')
    
    print('---' * 25)
    

             y_test    y_pred
loinc_num                    
1250-0     0.000000  0.371335
13317-3    0.308807  0.323812
1975-2     0.362224  0.411951
2075-0     0.388614  0.317757
20442-0    0.290315  0.353132
6768-6     0.348299  0.442653
17861-6    0.388614  0.405318
890-4      0.465409  0.436935
18928-2    0.000000  0.396889
15076-3    0.619130  0.210901
Model for query = "glucose in blood" obtains an error: 0.20
---------------------------------------------------------------------------
             y_test    y_pred
loinc_num                    
3082-5     0.617614  0.415872
10331-7    0.555283  0.201716
18906-8    0.000000  0.277435
1968-7     0.586489  0.549840
2132-9     0.377964  0.464527
1003-3     0.478651  0.491005
54439-5    0.508672  0.514453
18865-6    0.000000  0.375757
1995-0     0.525473  0.285851
18878-9    0.000000  0.297970
Model for query = "bilirubin in plasma" obtains an error: 0.24
---------------------------------------------------------------------------
     